# Space Laser v9 - Pretraining On The SOC Manual

In [1]:
# Three Sections: (at the moment)
# 1) Installation Commands
# 2) Initialization Cells
# 3) Training Loop

print("Hooray! Here we go!")

# lolol

Hooray! Here we go!


# Install Commands

In [ ]:
# Consolidated
conda install pandas scikit-learn transformers pytorch torchvision torchaudio cudatoolkit=11.8 -c pytorch -c conda-forge -c nvidia -y

In [2]:
conda install pandas -y

Note: you may need to restart the kernel to use updated packages.


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/win-64/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/win-64/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/noarch/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/msys2/win-64/current_repodata.json HTTP/1.1" 304 0
DEBUG:


Solving environment: ...working... done

# All requested packages already installed.



In [ ]:
conda install scikit-learn -y

In [ ]:
conda install -c conda-forge tensorboard

In [ ]:
pip install datetime

In [ ]:
conda install transformers -y

In [ ]:
conda install pytorch torchvision torchaudio cudatoolkit=11.8 -c pytorch -c nvidia

# Initialization Cells

In [3]:
#CELL A: Load and Split the Data

import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv("C://Offline_Storage//radiantClass//JobDesc_BlobText.csv")

# Split the data
train, temp = train_test_split(data, test_size=0.1, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)

In [6]:
#CELL B: Setting up the Model and Transformer

from transformers import BertTokenizer, BertForMaskedLM
from transformers import DataCollatorForLanguageModeling

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

# Data collator will help with constructing batches for training
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm_probability=0.15  # 15% of tokens will be masked for prediction
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
#CELL C: Tokenize the Data

train_encodings = tokenizer(list(train['JobDesc_BlobText']), truncation=True, padding='max_length', max_length=512)
val_encodings = tokenizer(list(val['JobDesc_BlobText']), truncation=True, padding='max_length', max_length=512)
test_encodings = tokenizer(list(test['JobDesc_BlobText']), truncation=True, padding='max_length', max_length=512)

In [8]:
#CELL D: Convert to Dataset and DataLoader

from torch.utils.data import Dataset, DataLoader

class JobTitleDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = JobTitleDataset(train_encodings)
val_dataset = JobTitleDataset(val_encodings)
test_dataset = JobTitleDataset(test_encodings)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=8, collate_fn=data_collator)
val_loader = DataLoader(val_dataset, batch_size=8, collate_fn=data_collator)

In [11]:
#CELL E: Training Setup and Device Configuration
import torch
from torch.optim import AdamW

# Check for CUDA and move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Setup optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

In [13]:
import torch
print(f"Is Cuda available?: {torch.cuda.is_available()}")

Is Cuda available?: False


In [12]:
#CELL F: Verify that CUDA is available

print(f"Is Cuda available?: {torch.cuda.is_available()}! :-)")
print(f"CUDA version: {torch.version.cuda}")

Is Cuda available?: False! :-)
CUDA version: None


In [ ]:
#CELL Proto-G:

%load_ext tensorboard

In [ ]:
#CELL G: Set Up TensorBoard within the Notebook

%tensorboard --logdir ./logs

# Training Loop

In [ ]:
#CELL H: Model Training Using Trainer

from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=8,  # Adjust batch size based on your GPU/CPU capacity
    per_device_eval_batch_size=8,  # Same for validation batch size
    num_train_epochs=3,  # Adjust the number of epochs based on your needs
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    do_train=True,
    do_eval=True,
    save_strategy="epoch",  # Save only the best model based on validation loss
    load_best_model_at_end=True,  # Load the best model at the end of training
    push_to_hub=False,  # Disable model hub push
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator  # Use the data_collator from CELL B
)

# Start training
trainer.train()
